In [1]:
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import pandas as pd
from sklearn.datasets import dump_svmlight_file

In [2]:
def preprocess(df, train):
    feature_labels = df.columns.values.tolist()

    # we remove the features where more than 50% of the data is missing from the training set
    remove = []
    for n in range(1, 9):
        remove += ["comp%d_inv" % n, "comp%d_rate" % n, "comp%d_rate_percent_diff" % n]
    remove += ["srch_query_affinity_score", "visitor_hist_adr_usd", "visitor_hist_starrating"]

    if "position" in feature_labels:
        remove += ["position", "click_bool", "booking_bool", "gross_bookings_usd"]  # training set only

    remove += ["srch_id", "prop_id"]

    for l in remove:
        feature_labels.remove(l)

    # outliers in hotel price, hotels with price > 10000 are removed from training set. Source: David Wind
    if train:
        df = df[df["price_usd"] < 10000]

    #######################
    # FEATURE ENGINEERING #
    #######################

    # From paper "Combination of Diverse Ranking Models for Personalized Expedia Hotel Searches"
    df["count_window"] = df["srch_room_count"] * max(df["srch_booking_window"]) + df["srch_booking_window"]
    feature_labels.append("count_window")

    # month, week, day of the week and hour of search
    df_datetime = pd.DatetimeIndex(df.date_time)
    df["month"] = df_datetime.month
    df["week"] = df_datetime.week
    df["day"] = df_datetime.dayofweek + 1
    df["hour"] = df_datetime.hour
    feature_labels += ["month", "week", "day", "hour"]
    feature_labels.remove("date_time")

    # sanitize prop_log_historical_price
    df["prop_historical_price"] = (np.e ** df["prop_log_historical_price"]).replace(1.0, 10000)
    feature_labels.append("prop_historical_price")
    feature_labels.remove("prop_log_historical_price")

    features = df[feature_labels].values
    qid = df['srch_id'].values
    target = np.zeros(len(df))
    if 'booking_bool' in df.columns.values.tolist():
        target = np.fmax((5 * df['booking_bool']).values, df['click_bool'].values)

    return df, features, qid, target, feature_labels

In [3]:
nrows = None
#nrows = int(1e5)
data_train = pd.read_csv("training_set_VU_DM.csv", header=0, parse_dates=[1], nrows=nrows)
data_test = pd.read_csv("test_set_VU_DM.csv", header=0, parse_dates=[1], nrows=nrows)

print("loaded csv files")

# pre-fill missing prop_location_score2 scores with first quartile of country:
# source Bing Xu et al (fourth place)
all_data = pd.concat([data_train, data_test], copy=False)
location_quartile = all_data.groupby("prop_country_id")["prop_location_score2"].quantile(q=0.25)

for d in (data_train, data_test):
    d["prop_location_score2_quartile"] = location_quartile[d.prop_id].values
    d["prop_location_score2"].fillna(d["prop_location_score2_quartile"])
    del d["prop_location_score2_quartile"]


# fill missing values with worst case scenario. Source: Jun Wang 3rd place
# ["prop_review_score", "prop_location_score2", "orig_destination_distance"]
data_train = data_train.fillna(value=-1)
data_test = data_test.fillna(value=-1)

# feature engineering using all numeric features
# avg/median/std numeric features per prop_id
numeric_features = ["prop_starrating", "prop_review_score", "prop_location_score1", "prop_location_score2"]
all_data = pd.concat([data_train, data_test], copy=False)

for label in numeric_features:
    mean = all_data.groupby("prop_id")[label].mean().fillna(value=-1)
    median = all_data.groupby("prop_id")[label].median().fillna(value=-1)
    std = all_data.groupby("prop_id")[label].std().fillna(value=-1)

    for d in (data_train, data_test):
        d[label + "_mean"] = mean[d.prop_id].values
        d[label + "_median"] = median[d.prop_id].values
        d[label + "_std"] = std[d.prop_id].values

train, Xtr, qtr, ytr, feature_labels = preprocess(data_train[data_train.srch_id % 10 != 0], train=True)
print("preprocessed training data")
vali, Xva, qva, yva, feature_labels = preprocess(data_train[data_train.srch_id % 10 == 0], train=True)
del data_train
print("preprocessed validation data")
test, Xte, qte, yte, feature_labels = preprocess(data_test, train=False)
print("preprocessed test data")
del data_test

comment = ' '.join(map(lambda t: '%d:%s' % t, zip(range(len(feature_labels)), feature_labels)))
print('Features:')
print(comment)

loaded csv files


c:\users\matev\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:10: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas will change to not sort by default.

To accept the future behavior, pass 'sort=False'.

To retain the current behavior and silence the warning, pass 'sort=True'.

  # Remove the CWD from sys.path while we load stuff.
c:\users\matev\appdata\local\programs\python\python37\lib\site-packages\pandas\core\series.py:942: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self.loc[key]
c:\users\matev\appdata\local\programs\python\python37\lib\site-packages\ipykernel_launcher.py:27: FutureWarning: Sorting because non-concatenation axis is not aligned. A future version
of pandas wil

preprocessed training data
preprocessed validation data
preprocessed test data
Features:
0:site_id 1:visitor_location_country_id 2:prop_country_id 3:prop_starrating 4:prop_review_score 5:prop_brand_bool 6:prop_location_score1 7:prop_location_score2 8:price_usd 9:promotion_flag 10:srch_destination_id 11:srch_length_of_stay 12:srch_booking_window 13:srch_adults_count 14:srch_children_count 15:srch_room_count 16:srch_saturday_night_bool 17:orig_destination_distance 18:random_bool 19:prop_starrating_mean 20:prop_starrating_median 21:prop_starrating_std 22:prop_review_score_mean 23:prop_review_score_median 24:prop_review_score_std 25:prop_location_score1_mean 26:prop_location_score1_median 27:prop_location_score1_std 28:prop_location_score2_mean 29:prop_location_score2_median 30:prop_location_score2_std 31:count_window 32:month 33:week 34:day 35:hour 36:prop_historical_price


In [8]:
print(feature_labels)

['site_id', 'visitor_location_country_id', 'prop_country_id', 'prop_starrating', 'prop_review_score', 'prop_brand_bool', 'prop_location_score1', 'prop_location_score2', 'price_usd', 'promotion_flag', 'srch_destination_id', 'srch_length_of_stay', 'srch_booking_window', 'srch_adults_count', 'srch_children_count', 'srch_room_count', 'srch_saturday_night_bool', 'orig_destination_distance', 'random_bool', 'prop_starrating_mean', 'prop_starrating_median', 'prop_starrating_std', 'prop_review_score_mean', 'prop_review_score_median', 'prop_review_score_std', 'prop_location_score1_mean', 'prop_location_score1_median', 'prop_location_score1_std', 'prop_location_score2_mean', 'prop_location_score2_median', 'prop_location_score2_std', 'count_window', 'month', 'week', 'day', 'hour', 'prop_historical_price']


In [4]:
def dump(args):
    """Dumps to svmlight format."""
    x, y, filename, query_id, comment = args
    dump_svmlight_file(x, y, filename, query_id=query_id, comment=comment, zero_based=False)

dump((Xtr, ytr, 'train.svmlight', qtr, comment))
print('Dumped train data')
dump((Xva, yva, 'vali.svmlight', qva, comment))
print('Dumped val data')
dump((Xte, yte, 'test.svmlight', qte, comment))
print('Dumped test data')

Dumped train data
Dumped val data
Dumped test data


In [5]:
# This cell is ran independendly to get full train set in svmlight without validation

nrows = None
data_train = pd.read_csv("training_set_VU_DM.csv", header=0, parse_dates=[1], nrows=nrows)
data_test = pd.read_csv("test_set_VU_DM.csv", header=0, parse_dates=[1], nrows=nrows)

print("loaded csv files")

# pre-fill missing prop_location_score2 scores with first quartile of country:
# source Bing Xu et al (fourth place)
all_data = pd.concat([data_train, data_test], copy=False)
location_quartile = all_data.groupby("prop_country_id")["prop_location_score2"].quantile(q=0.25)

for d in (data_train, data_test):
    d["prop_location_score2_quartile"] = location_quartile[d.prop_id].values
    d["prop_location_score2"].fillna(d["prop_location_score2_quartile"])
    del d["prop_location_score2_quartile"]

# fill missing values with worst case scenario. Source: Jun Wang 3rd place
# ["prop_review_score", "prop_location_score2", "orig_destination_distance"]
data_train = data_train.fillna(value=-1)
data_test = data_test.fillna(value=-1)

# feature engineering using all numeric features
# avg/median/std numeric features per prop_id
numeric_features = ["prop_starrating", "prop_review_score", "prop_location_score1", "prop_location_score2"]
all_data = pd.concat([data_train, data_test], copy=False)

for label in numeric_features:
    mean = all_data.groupby("prop_id")[label].mean().fillna(value=-1)
    median = all_data.groupby("prop_id")[label].median().fillna(value=-1)
    std = all_data.groupby("prop_id")[label].std().fillna(value=-1)

    for d in (data_train, data_test):
        d[label + "_mean"] = mean[d.prop_id].values
        d[label + "_median"] = median[d.prop_id].values
        d[label + "_std"] = std[d.prop_id].values

train, Xtr, qtr, ytr, feature_labels = preprocess(data_train, train=True)
print("preprocessed training data")
comment = ' '.join(map(lambda t: '%d:%s' % t, zip(range(len(feature_labels)), feature_labels)))
dump((Xtr, ytr, 'train_full.svmlight', qtr, comment))
print('Dumped full train data')

KeyboardInterrupt: 